In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import datetime as dt
import requests
import json
import base64
import hmac
import hashlib
import time
import asyncio
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [ ]:
from coinbase.wallet.client import Client
#public_client = cbpro.PublicClient()
import cbpro
public_client = cbpro.PublicClient()

In [ ]:
""" loading the client's api keys and secret keys """
load_dotenv()
api_key = os.getenv("COINBASE_SAND_API")
api_secret = os.getenv("COINBASE_SAND_SECRET")
#api_passphrase = "columbia2021"
api_passphrase = os.getenv("COINBASE_SAND_PASSPHRASE")
#print(api_passphrase)
#print(api_secret)
#client = Client(api_key, api_secret, api_passphrase)


In [ ]:
public_client = cbpro.PublicClient()

In [ ]:
""" authorizing the api to interact with the sandbox"""

auth_client = cbpro.AuthenticatedClient(api_key, api_secret, api_passphrase,
                                  api_url="https://api-public.sandbox.pro.coinbase.com")

In [ ]:
data = public_client.get_product_historic_rates('BTC-USD')
data

In [ ]:
accounts = auth_client.get_accounts()
accounts

In [ ]:
""" authorizing the api to interact with the sandbox"""

auth_client = cbpro.AuthenticatedClient(api_key, api_secret, api_passphrase,
                                  api_url="https://api-public.sandbox.pro.coinbase.com")

In [ ]:


#requests.get("https://api-public.sandbox.exchange.coinbase.com")



In [ ]:
currency_code = "USD"

In [ ]:
currencies = public_client.get_product_ticker(201)
#btc = currencies[201]
#ada = currencies[35]
currencies

In [ ]:
public_client.get_product_order_book('BTC-USD')

In [ ]:
btc_ticker = public_client.get_product_ticker(product_id='BTC-USD')
#btc_ticker_df = pd.DataFrame(btc_ticker)

In [ ]:
""" getting 24 hour stats """

btc_24hr = public_client.get_product_24hr_stats('BTC-USD')

btc_24hr_df = pd.DataFrame.from_dict(btc_24hr, 
    orient='index',dtype=None, #columns=None
)

btc_24hr_df


In [ ]:
""" getting the product ticker and time stamp """

btc = public_client.get_product_ticker('BTC-USD')

btc_df = pd.DataFrame.from_dict(btc,
    orient='index',dtype=None, #columns=None
)

btc_df

In [ ]:
""" resetting the dataframe and time as index, dropping unneeded columns"""

btc_close = btc_df.transpose().set_index('time')

#btc_close = btc_df.transpose().set_index('time').drop(columns=['trade_id', 'size', 'ask', 'volume', 'bid'])


btc_close

In [ ]:
#btc_close['time'] = pd.to_datetime(btc_close['time'], unit='d')

In [ ]:
""" getting the client's sandbox wallet """
auth_client.get_accounts()

In [ ]:
""" sell order btc to usd """

auth_client.sell(size='1', #BTC
               order_type='market',
               product_id='BTC-USD')

In [ ]:
""" buy order btc from usd """
auth_client.buy(size='1', #BTC
               order_type='market',
               product_id='BTC-USD')

next is setting the buy and sell signals

In [118]:
btc_close

,trade_id,price,size,bid,ask,volume
time,,,,,,
2021-10-11T13:35:46.111273Z,219964318,56889.99,0.00050915,56889.99,56890,10914.66157561


In [ ]:
btc_hist_data = public_client.get_product_historic_rates('BTC-USD')
btc_hist_data

In [ ]:
import btalib
""" not sure if i need this import """

In [ ]:
""" converting the historical data to dataframe and renaming the columns
!!! I need to change the time columns to datetime and don't know how !!!
"""
btc_hist_data_df = pd.DataFrame(data, columns=['date', 'open', 'high', 'low', 'close', 'volume'])
btc_hist_data_df .set_index('date', inplace=True)
print(btc_hist_data_df.head())

In [ ]:
""" saving the historical data to csv """
btc_hist_data_df.to_csv('HistoricRates')


In [ ]:
#btc_hist_data_df['20sma'] = dataframe.close.rolling(20).mean()
#print(btc_hist_data_df.tail(5))

In [ ]:
#btc_sma = btalib.sma(btc_hist_data_df.close)
#print(btc_sma.df)

In [ ]:
#btc_hist_data_df['sma'] = btalib.sma(btc_hist_data_df.close, period=20).df
#btc_hist_data_df.tail()

In [ ]:
""" creating a signals dataframe to hole the values of the actual returns """

signals_df = btc_hist_data_df.loc[:, ["close"]]

signals_df["Actual Returns"] = signals_df["close"].pct_change()

signals_df = signals_df.dropna()

display(signals_df.head())
display(signals_df.tail())

In [ ]:
""" setting the rolling windows for the long and short positions"""
short_window = 4
long_window = 100

signals_df['SMA_Fast'] = signals_df['close'].rolling(window=short_window).mean()
signals_df['SMA_Slow'] = signals_df['close'].rolling(window=long_window).mean()

signals_df = signals_df.dropna()

display(signals_df.head())
display(signals_df.tail())

In [ ]:
""" creating a new signals dataframe to hold the signals
creating buy and sell signals according to the actual returns in the data
"""
signals_df['Signal'] = 0.0

signals_df.loc[(signals_df['Actual Returns'] >= 0), 'Signal'] = 1

signals_df.loc[(signals_df['Actual Returns'] < 0), 'Signal'] = -1

display(signals_df.head())
display(signals_df.tail())

In [ ]:
""" evaluating the balance of the signals"""
signals_df['Signal'].value_counts()

In [ ]:
""" creating a strategy returns colums to hold the returns data using the signals
programmed in the code
"""
signals_df['Strategy Returns'] = signals_df['Actual Returns'] * signals_df['Signal'].shift()

display(signals_df.head())
display(signals_df.tail())

In [ ]:
""" plotting the strategy returns to better see the data """

(1 + signals_df['Strategy Returns']).cumprod().plot(x="close")

In [ ]:
""" creating an X features dataframe to split the data into training and testing data"""

X = signals_df[['SMA_Fast', 'SMA_Slow']].shift().dropna()


X.head()

In [ ]:
""" creating a y dataframe to hold the prediction data """

y = signals_df['Signal']


y.value_counts()

In [ ]:
""" telling the algorithm when to start the training data """
training_begin = X.index.min()

print(training_begin)

In [ ]:
""" telling the algorithm when to end the training data """
training_end = X.index.min() + DateOffset(months=3)

print(training_end)

In [ ]:
""" creating new training dataframes using the start/end data"""
X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

X_train.head()

In [ ]:
""" creating new testing dataframes using the start/end data"""
X_test = X.loc[training_end+DateOffset(hours=1):]
y_test = y.loc[training_end+DateOffset(hours=1):]

X_test.head()

In [ ]:
""" scaling the X data using StandardScaler instance """

scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
""" creating an SVM model to backtest the data """

svm_model = svm.SVC()
 
svm_model = svm_model.fit(X_train_scaled, y_train)
 
svm_pred = svm_model.predict(X_test_scaled)

svm_pred[:10]

In [ ]:
""" creating a classification report to see the model's performance """
svm_testing_report = classification_report(y_test, svm_pred)


print(svm_testing_report)

In [ ]:
""" creating a new dataframe to hold the predictions of y using the X_test data 
We are using the actual and predicted returns to calculate the strategy returns
"""

predictions_df = pd.DataFrame(index=X_test.index)


predictions_df['Predicted'] = svm_pred


predictions_df['Actual Returns'] = signals_df['Actual Returns']


predictions_df['Strategy Returns'] = predictions_df['Actual Returns'] * predictions_df['Predicted']

display(predictions_df.head())
display(predictions_df.tail())

In [ ]:
""" plotting the actual returns vs the strategy returns to analyse the performance of the model"""


(1 + predictions_df[["Actual Returns", "Strategy Returns"]]).cumprod().plot()